## This notebook compiles covid 19 and heart attack data

In [ ]:
import pandas as pd

import geopandas as gpd

county_dict = {"Bronx":	"Bronx County", "Brooklyn":	"Kings County", "Manhattan":	"New York County", "Queens":	"Queens County", "Staten Island":	"Richmond County"}

In [ ]:
# SHP to CSV

def shp_to_csv(shapefile_path, csv_output_path):
  gdf = gpd.read_file(shapefile_path)
  gdf.to_csv(csv_output_path, index=False)


# The folder containing the .shp file should also contain the following files: .cpg, .dbf, .prg, ise.xml, .iso.xml, .shx
shapefile_path = 'tl_2020_36_tabblock10/tl_2020_36_tabblock10.shp'
csv_output_path = 'tl_2020_36_tabblock10.csv'
shp_to_csv(shapefile_path, csv_output_path)

# FIP codes

In [ ]:
# Desc: This method reads FIPS data from a file, processes it to extract state and county codes, and filters records for the state code '36' (New York) before returning the result.
# O/p: fips (DataFrame): A pandas DataFrame containing FIPS data with columns 'FIP', 'countyName', 'stateFP', and 'countyFP'. The data is filtered to include only records for the state with the code '36' (New York).

# def getFIPS():
#   fips = pd.read_csv('fips.txt', sep=' ' * 8, header=None, names=['FIP', 'countyName'], skiprows=72, error_bad_lines=False)

#   fips['stateFP'] = fips['FIP'] // 1000
#   fips['countyFP'] = fips['FIP'] % 1000

#   fips['countyFP'] = fips['countyFP'].astype(int)

#   fips['stateFP'] = fips['stateFP'].apply(lambda x: f'{x:02d}')
#   fips['countyFP'] = fips['countyFP'].apply(lambda x: f'{x:03d}')

#   filtered_fips = fips[fips['stateFP'] == '36']
#   return filtered_fips

def getFIPS():
    fips = pd.read_csv('fips.txt', sep=' ' * 8, header=None, names=['FIP', 'countyName'], skiprows=72, error_bad_lines=False)

    fips['stateFP'] = fips['FIP'] // 1000
    fips['countyFP'] = fips['FIP'] % 1000

    fips['countyFP'] = fips['countyFP'].astype(int)

    fips['stateFP'] = fips['stateFP'].apply(lambda x: f'{x:02d}')
    fips['countyFP'] = fips['countyFP'].apply(lambda x: f'{x:03d}')

    # Filter based on county_dict
    filtered_fips = fips[fips['countyName'].isin(county_dict.values())]

    return filtered_fips


getFIPS()


<ipython-input-39-650d529636c1>:19: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  fips = pd.read_csv('fips.txt', sep=' ' * 8, header=None, names=['FIP', 'countyName'], skiprows=72, error_bad_lines=False)
<ipython-input-39-650d529636c1>:19: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  fips = pd.read_csv('fips.txt', sep=' ' * 8, header=None, names=['FIP', 'countyName'], skiprows=72, error_bad_lines=False)
Skipping line 147: Expected 2 fields in line 147, saw 5. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 162: Expected 2 fields in line 162, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping l

,FIP,countyName,stateFP,countyFP
201,6031,Kings County,06,031
512,13245,Richmond County,13,245
1858,36005,Bronx County,36,005
1879,36047,Kings County,36,047
1886,36061,New York County,36,061
1896,36081,Queens County,36,081
1898,36085,Richmond County,36,085
1995,37153,Richmond County,37,153
2937,51159,Richmond County,51,159


# Tiger Dataset

In [ ]:
# Desc: Reads FIPS data from a text file, extracts state and county information, filters for New York (stateFP = 36), and returns a DataFrame.
# O/p: filtered_fips (DataFrame): A pandas DataFrame containing FIPS data for New York state (stateFP = 36). Columns include 'FIP', 'countyName', 'stateFP', and 'countyFP'.

def processTiger(csv_file_path):
  tiger = pd.read_csv(csv_file_path)
  tiger = tiger.drop(columns=['MTFCC20', 'UR20', 'UACE20', 'UATYPE20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'HOUSING20', 'NAME20', 'POP20', 'MTFCC10', 'UR10', 'UACE10', 'UATYPE', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'NAME10'], axis=1, errors='ignore')
  tiger = tiger.assign(**{'COUNTYFP10': tiger['COUNTYFP10'].apply(lambda x: f'{x:03d}')} if 'COUNTYFP10' in tiger.columns else {})
  tiger = tiger.assign(**{'COUNTYFP20': tiger['COUNTYFP20'].apply(lambda x: f'{x:03d}')} if 'COUNTYFP20' in tiger.columns else {})
  return tiger

# tiger = processTiger('tl_2020_36_tabblock10.csv')
tiger = processTiger('tl_2020_36_tabblock20.csv')

tiger

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,INTPTLAT20,INTPTLON20
0,36,083,51702,1049,360830517021049,42.897125,-73.352272
1,36,073,40603,3033,360730406033033,43.215634,-78.304855
2,36,017,970501,1061,360179705011061,42.619681,-75.418301
3,36,017,970602,3011,360179706023011,42.466263,-75.425369
4,36,103,123500,2010,361031235002010,40.686892,-73.422595
...,...,...,...,...,...,...,...
288814,36,025,971300,1093,360259713001093,41.970414,-75.026552
288815,36,025,970800,2048,360259708002048,42.240209,-74.935034
288816,36,109,1902,1014,361090019021014,42.363794,-76.497991
288817,36,071,1602,2014,360710016022014,41.436885,-74.432279


# COVID-19

In [ ]:
# Desc: Processes COVID hospitalization rate data, calculates average rates by year for each borough, maps county names, merges with FIPS data, and aggregates by countyFP.
# I/p: csv_file_path (str): Path to the CSV file containing COVID hospitalization rate data.
# O/p: COVID_hospperyear (DataFrame): A pandas DataFrame containing processed COVID hospitalization rate data. Columns include 'stateFP', 'countyFP', 'countyName', 'COVIDhosprate_2020', 'COVIDhosprate_2021', 'COVIDhosprate_2022', and 'COVIDhosprate_2023'.

def processCOVID(csv_file_path):
  hosprate_df = pd.read_csv(csv_file_path)
  columns_to_keep = ['date', 'HOSPRATE_Bronx', 'HOSPRATE_Brooklyn', 'HOSPRATE_Manhattan', 'HOSPRATE_Queens', 'HOSPRATE_Staten_Island']

  hosprate_filtered = hosprate_df[columns_to_keep]
  hosprate_filtered['date'] = pd.to_datetime(hosprate_filtered['date'], format='%m/%Y')

  average_hospitalizations_by_year = hosprate_filtered.groupby(hosprate_filtered['date'].dt.year).mean().iloc[:, :5]
  average_hospitalizations_by_year = average_hospitalizations_by_year.reset_index()

  melted_df = pd.melt(average_hospitalizations_by_year, id_vars=['date'], value_vars=['HOSPRATE_Bronx', 'HOSPRATE_Brooklyn', 'HOSPRATE_Manhattan', 'HOSPRATE_Queens', 'HOSPRATE_Staten_Island'], var_name='countyName', value_name='Hosprate')
  melted_df.rename(columns={'date': 'year'}, inplace=True)
  melted_df = melted_df[['year', 'Hosprate', 'countyName']]

  county_dict = {"Bronx":	"Bronx County", "Brooklyn":	"Kings County", "Manhattan":	"New York County", "Queens":	"Queens County", "Staten Island":	"Richmond County"}
  melted_df['countyName'] = melted_df['countyName'].str[9:].str.replace('_', ' ').replace(county_dict)

  fips = getFIPS()
  COVID_hospperyear = pd.merge(melted_df, fips, on='countyName', how='left')

  COVID_hospperyear['hosprate_2020'] = COVID_hospperyear.loc[COVID_hospperyear['year'] == 2020, 'Hosprate']
  COVID_hospperyear['hosprate_2021'] = COVID_hospperyear.loc[COVID_hospperyear['year'] == 2021, 'Hosprate']
  COVID_hospperyear['hosprate_2022'] = COVID_hospperyear.loc[COVID_hospperyear['year'] == 2022, 'Hosprate']
  COVID_hospperyear['hosprate_2023'] = COVID_hospperyear.loc[COVID_hospperyear['year'] == 2023, 'Hosprate']

  COVID_hospperyear = COVID_hospperyear.drop(columns=['year', 'FIP', 'Hosprate'], axis=1)

  COVID_hospperyear = COVID_hospperyear.groupby('countyFP', as_index=False).agg({
      'countyName': 'first',
      'stateFP': 'first',
      'hosprate_2020': 'sum',
      'hosprate_2021': 'sum',
      'hosprate_2022': 'sum',
      'hosprate_2023': 'sum'
  })

  COVID_hospperyear = COVID_hospperyear[['stateFP', 'countyFP', 'countyName', 'hosprate_2020', 'hosprate_2021', 'hosprate_2022', 'hosprate_2023']].rename(columns={'hosprate_2020': 'covid_hosprate_2020', 'hosprate_2021': 'covid_hosprate_2021', 'hosprate_2022': 'covid_hosprate_2022', 'hosprate_2023': 'covid_hosprate_2023'})

  return COVID_hospperyear


processedCOVID = processCOVID('hosprate-by-modzcta.csv')

processedCOVID

<ipython-input-18-b02ba226de86>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hosprate_filtered['date'] = pd.to_datetime(hosprate_filtered['date'], format='%m/%Y')
<ipython-input-18-b02ba226de86>:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_hospitalizations_by_year = hosprate_filtered.groupby(hosprate_filtered['date'].dt.year).mean().iloc[:, :5]
<ipython-input-12-00f2e2a693fb>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are inter

,stateFP,countyFP,countyName,covid_hosprate_2020,covid_hosprate_2021,covid_hosprate_2022,covid_hosprate_2023
0,36,005,Bronx County,100.33,73.616667,73.058333,25.43
1,36,047,Kings County,76.81,61.633333,54.350000,17.88
2,36,061,New York County,57.41,42.966667,49.475000,18.67
3,36,081,Queens County,91.46,60.933333,52.616667,18.00
4,36,085,Richmond County,75.53,64.633333,66.091667,22.45


In [ ]:
tiger = processTiger('tl_2020_36_tabblock20.csv')

In [ ]:
# Desc: Merges TIGER demographic data with processed COVID hospitalization rate data based on countyFP.
# I/p: tiger (DataFrame): TIGER demographic data DataFrame.
# I/p: processedCOVID (DataFrame): Processed COVID hospitalization rate data DataFrame.
# O/p: tiger_COVID_merged (DataFrame): Merged DataFrame containing TIGER demographic data and processed COVID hospitalization rate data.

def mergeCOVID(tiger, processedCOVID):
  tiger_COVID_merged = pd.merge(tiger, processedCOVID, left_on='COUNTYFP20', right_on='countyFP', how='inner')

  tiger_COVID_merged = tiger_COVID_merged.drop(columns=['stateFP', 'countyFP'], axis=1)

  return tiger_COVID_merged

tiger_COVID_merged = mergeCOVID(tiger, processedCOVID)

tiger_COVID_merged

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,INTPTLAT20,INTPTLON20,countyName,covid_hosprate_2020,covid_hosprate_2021,covid_hosprate_2022,covid_hosprate_2023
0,36,081,65600,1009,360810656001009,40.663372,-73.732427,Queens County,91.46,60.933333,52.616667,18.00
1,36,081,15500,1002,360810155001002,40.759207,-73.916049,Queens County,91.46,60.933333,52.616667,18.00
2,36,081,118700,1002,360811187001002,40.760558,-73.811251,Queens County,91.46,60.933333,52.616667,18.00
3,36,081,99100,5000,360810991005000,40.793713,-73.800569,Queens County,91.46,60.933333,52.616667,18.00
4,36,081,24000,1001,360810240001001,40.702762,-73.808812,Queens County,91.46,60.933333,52.616667,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...
37979,36,085,29102,2020,360850291022020,40.580204,-74.204633,Richmond County,75.53,64.633333,66.091667,22.45
37980,36,085,19800,4032,360850198004032,40.521445,-74.187190,Richmond County,75.53,64.633333,66.091667,22.45
37981,36,085,19800,1011,360850198001011,40.507198,-74.216702,Richmond County,75.53,64.633333,66.091667,22.45
37982,36,085,11201,2,360850112010002,40.572261,-74.075894,Richmond County,75.53,64.633333,66.091667,22.45


In [ ]:
tiger_COVID_merged.to_excel('COVID_hosprate_by_year.xlsx' , index=  False)

# Heart Attack

In [ ]:
# Desc: Process CHIRS data for a specific county and indicator.
# I/p: allIndicators (DataFrame): DataFrame containing all CHIRS indicators.
# I/p: countyName (str): Name of the county for which the data should be processed.
# O/p: Returns a DataFrame containing processed CHIRS data for heart attack hospitalization rate per 10,000 for the specified county and indicator. Returns None if no data is found.


def process_CHIRS(allIndicators, countyName):
  NYC_indicators = allIndicators[allIndicators['Geographic area'] == countyName]

  indicator = 'Cardiovascular Disease Indicators'
  filtered_df = NYC_indicators[(NYC_indicators['Topic Area'] == indicator) &
                                  (NYC_indicators['Indicator Title'] == 'Heart attack hospitalization rate per 10,000') &
                                  (~NYC_indicators['Year'].str.contains('-'))]

  filtered_df['Rate/Percent'] = filtered_df['Rate/Percent'].astype(float) / 10

  if not filtered_df.empty:
    HA_final = filtered_df.pivot(index='Geographic area', columns='Year', values='Rate/Percent').reset_index()
    HA_final.columns.name = None  # Remove the 'Year' label from the columns
    HA_final.columns = ['Geographic Area'] + [f'cardio_{year}' for year in HA_final.columns[1:]]
    return HA_final
  else:
    return None

# Desc: Process Heart Attack (HA) data for multiple counties
# I/p: csv_file_path (str): Path to the CSV file containing CHIRS indicators.
# O/p: Returns a DataFrame containing processed Heart Attack (HA) data for multiple counties, including FIPS information.

def processHA(csv_file_path):
  NY_indicators = pd.read_excel(csv_file_path, sheet_name=1)

  results_list = []

  for county_name in county_dict.values():
    result = process_CHIRS(NY_indicators, county_name)
    results_list.append(result)

  HA_final = pd.concat(results_list, ignore_index=True)

  fips = getFIPS()

  HA_final = pd.merge(HA_final, fips, left_on='Geographic Area' ,right_on='countyName', how='left')

  HA_final = HA_final[['stateFP', 'countyFP', 'countyName', 'cardio_2011', 'cardio_2012', 'cardio_2013', 'cardio_2014', 'cardio_2016', 'cardio_2017', 'cardio_2018', 'cardio_2019', 'cardio_2020']]


  return HA_final




HA_filePath = 'NewYork_chir_county_trend.xlsx'
processedHA = processHA(HA_filePath)

processedHA

<ipython-input-35-4cb37cb719e4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rate/Percent'] = filtered_df['Rate/Percent'].astype(float) / 10
<ipython-input-35-4cb37cb719e4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rate/Percent'] = filtered_df['Rate/Percent'].astype(float) / 10
<ipython-input-35-4cb37cb719e4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,stateFP,countyFP,countyName,cardio_2011,cardio_2012,cardio_2013,cardio_2014,cardio_2016,cardio_2017,cardio_2018,cardio_2019,cardio_2020
0,36,005,Bronx County,1.56,1.41,1.35,1.24,1.29,1.29,1.31,1.36,1.12
1,36,047,Kings County,1.59,1.59,1.50,1.47,1.44,1.31,1.41,1.40,1.03
2,36,061,New York County,1.04,1.14,1.10,1.07,1.01,0.98,1.19,1.15,0.80
3,36,081,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
4,36,085,Richmond County,1.73,1.50,1.53,1.66,1.84,1.73,1.59,1.59,1.29


In [ ]:
tiger = processTiger('tl_2020_36_tabblock20.csv')

In [ ]:
# Desc: Merge Tiger data with processed Heart Attack (HA) data
# I/p: tiger (pd.DataFrame): Tiger dataset.
# I/p: processedHA (pd.DataFrame): Processed Heart Attack (HA) data.
# O/p: Returns a DataFrame resulting from merging Tiger data with processed Heart Attack (HA) data. The unnecessary stateFP and countyFP columns are dropped in the process.

def mergeHA(tiger, processedHA):
  tiger_HA_merged = pd.merge(tiger, processedHA, left_on='COUNTYFP20', right_on='countyFP', how='inner')

  tiger_HA_merged = tiger_HA_merged.drop(columns=['stateFP', 'countyFP'], axis=1)

  tiger_HA_merged

  return tiger_HA_merged

tiger_HA_merged = mergeHA(tiger, processedHA)

tiger_HA_merged

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,INTPTLAT20,INTPTLON20,countyName,cardio_2011,cardio_2012,cardio_2013,cardio_2014,cardio_2016,cardio_2017,cardio_2018,cardio_2019,cardio_2020
0,36,081,65600,1009,360810656001009,40.663372,-73.732427,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
1,36,081,15500,1002,360810155001002,40.759207,-73.916049,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
2,36,081,118700,1002,360811187001002,40.760558,-73.811251,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
3,36,081,99100,5000,360810991005000,40.793713,-73.800569,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
4,36,081,24000,1001,360810240001001,40.702762,-73.808812,Queens County,1.51,1.48,1.40,1.42,1.51,1.47,1.45,1.54,1.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37979,36,085,29102,2020,360850291022020,40.580204,-74.204633,Richmond County,1.73,1.50,1.53,1.66,1.84,1.73,1.59,1.59,1.29
37980,36,085,19800,4032,360850198004032,40.521445,-74.187190,Richmond County,1.73,1.50,1.53,1.66,1.84,1.73,1.59,1.59,1.29
37981,36,085,19800,1011,360850198001011,40.507198,-74.216702,Richmond County,1.73,1.50,1.53,1.66,1.84,1.73,1.59,1.59,1.29
37982,36,085,11201,2,360850112010002,40.572261,-74.075894,Richmond County,1.73,1.50,1.53,1.66,1.84,1.73,1.59,1.59,1.29


In [ ]:
tiger_HA_merged.to_excel('HeartAttacks_hosprate_by_year.xlsx' , index=  False)